# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `local_grader.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder
from scipy.stats import ttest_ind


# Modify this to point to your data.
unique_data = pd.read_csv('data.txt', sep = "\t")
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11,col_12
0,3,-324 kg,0.7957,robotics engineering,Rat,1157,basketball,NaN,526,BIRD,1.2592,-318,1017,aNAHEIM
1,5,841 kg,1.1915,Bioengineering,reptile,678,golf,1.1419,1521,Guinea Pig,0.6987,983,1025,San Francisco
2,5,818 kg,1.1911,Bioengineering,reptile,437,NaN,0.7436,946,guinea pig,0.6277,513,1066,Long Beach
3,4,147 kg,0.4575,Computational Media,reptile,1147,tennis,0.3077,803,cat,1.289,338,988,Sacramento
4,0,686 kg,0.7226,Biotechnology,dog,966,boxing,1.2564,1028,chicken,1.0412,715,941,Oakland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,1,1149 kg,1.0949,Statistics,cat,1072,football,0.8021,1273,reptile,0.6317,148,457,Chula Vista
1212,1,1047 kg,0.2041,Statistics,cat,863,football,1.1701,355,reptile,0.4061,76,558,Long Beach
1213,1,484 kg,0.633,Statistics,dog,1311,football,1.1667,1281,reptile,1.0689,319,549,Santa Clarita
1214,0,1017 kg,1.4624,Computer Engineering,pig,1136,boxing,0.5672,1351,ferret,0.772,1010,883,Oakland


Don't forget to checkout the column information.

In [20]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216 entries, 0 to 1215
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1216 non-null   int64  
 1   col_00  1216 non-null   float64
 2   col_01  1216 non-null   float64
 3   col_02  1216 non-null   int64  
 4   col_03  1216 non-null   int64  
 5   col_04  1216 non-null   float64
 6   col_05  1216 non-null   int64  
 7   col_06  1216 non-null   float64
 8   col_07  1216 non-null   float64
 9   col_08  1216 non-null   int64  
 10  col_09  1216 non-null   float64
 11  col_10  1216 non-null   float64
 12  col_11  1216 non-null   float64
 13  col_12  1216 non-null   int64  
dtypes: float64(8), int64(6)
memory usage: 133.1 KB


And any numeric information.

In [21]:
unique_data.describe()

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11,col_12
count,1216.000000,1216.000000,1216.000000,1216.0,1216.0,1216.000000,1216.0,1216.000000,1216.000000,1216.0,1216.000000,1216.000000,1216.000000,1216.0
mean,2.513980,568.186678,0.689610,0.0,0.0,910.375822,0.0,0.865091,853.261513,0.0,0.762827,461.435855,871.254112,0.0
std,1.711679,476.170942,0.398474,0.0,0.0,345.237888,0.0,0.363810,434.638389,0.0,0.393856,499.005353,332.159551,0.0
min,0.000000,-897.000000,-0.661000,0.0,0.0,-393.000000,0.0,-0.393900,-708.000000,0.0,-0.581700,-792.000000,-400.000000,0.0
25%,1.000000,219.000000,0.431375,0.0,0.0,718.000000,0.0,0.646475,587.750000,0.0,0.522675,66.500000,672.750000,0.0
50%,2.000000,592.000000,0.690300,0.0,0.0,924.000000,0.0,0.898100,898.000000,0.0,0.837850,421.500000,920.000000,0.0
75%,4.000000,922.000000,0.966050,0.0,0.0,1131.000000,0.0,1.120500,1154.250000,0.0,1.045400,860.750000,1102.250000,0.0
max,5.000000,1663.000000,1.961500,0.0,0.0,1982.000000,0.0,2.035500,1981.000000,0.0,1.617000,1619.000000,1708.000000,0.0


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange; font-size: x-large";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [22]:
def clean_data(frame):
    frame = frame.copy()
    missing_values = ["N/A", "n/a", "NULL", "?", "None"]
    frame.replace(missing_values, np.nan, inplace=True)
    if "col_00" in frame.columns:
        frame["col_00"] = frame["col_00"].astype(str).str.replace(" kg", "", regex=True)
        frame["col_00"] = pd.to_numeric(frame["col_00"], errors="coerce")
    for col in frame.columns:
        if frame[col].dtype == 'object':
            frame[col] = pd.to_numeric(frame[col], errors='coerce')
    label_encoders = {}
    categorical_cols = ["col_02", "col_03", "col_05", "col_08", "col_12"]

    for col in categorical_cols:
        if col in frame.columns:
            frame[col] = frame[col].astype(str).str.lower().str.strip()
            label_encoders[col] = LabelEncoder()
            frame[col] = label_encoders[col].fit_transform(frame[col])
    frame.fillna(frame.median(numeric_only=True), inplace=True)

    return frame


unique_data = clean_data(unique_data)
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11,col_12
0,3,-324.0,0.7957,0,0,1157.0,0,0.8981,526.0,0,1.2592,-318.0,1017.0,0
1,5,841.0,1.1915,0,0,678.0,0,1.1419,1521.0,0,0.6987,983.0,1025.0,0
2,5,818.0,1.1911,0,0,437.0,0,0.7436,946.0,0,0.6277,513.0,1066.0,0
3,4,147.0,0.4575,0,0,1147.0,0,0.3077,803.0,0,1.2890,338.0,988.0,0
4,0,686.0,0.7226,0,0,966.0,0,1.2564,1028.0,0,1.0412,715.0,941.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,1,1149.0,1.0949,0,0,1072.0,0,0.8021,1273.0,0,0.6317,148.0,457.0,0
1212,1,1047.0,0.2041,0,0,863.0,0,1.1701,355.0,0,0.4061,76.0,558.0,0
1213,1,484.0,0.6330,0,0,1311.0,0,1.1667,1281.0,0,1.0689,319.0,549.0,0
1214,0,1017.0,1.4624,0,0,1136.0,0,0.5672,1351.0,0,0.7720,1010.0,883.0,0


Now we should also be able to view all the numeric columns.

In [23]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216 entries, 0 to 1215
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1216 non-null   int64  
 1   col_00  1216 non-null   float64
 2   col_01  1216 non-null   float64
 3   col_02  1216 non-null   int64  
 4   col_03  1216 non-null   int64  
 5   col_04  1216 non-null   float64
 6   col_05  1216 non-null   int64  
 7   col_06  1216 non-null   float64
 8   col_07  1216 non-null   float64
 9   col_08  1216 non-null   int64  
 10  col_09  1216 non-null   float64
 11  col_10  1216 non-null   float64
 12  col_11  1216 non-null   float64
 13  col_12  1216 non-null   int64  
dtypes: float64(8), int64(6)
memory usage: 133.1 KB


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [24]:
def create_classifiers():
    return [
        LogisticRegression(max_iter=1000),
        DecisionTreeClassifier(),
        KNeighborsClassifier(n_neighbors=5)
    ]

my_classifiers = create_classifiers()
my_classifiers

[LogisticRegression(max_iter=1000),
 DecisionTreeClassifier(),
 KNeighborsClassifier()]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `label` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [25]:
def cross_fold_validation(classifier, frame, folds):
    X = frame.drop(columns=["label"])
    y = frame["label"]
    X = X.fillna(X.median(numeric_only=True))
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    return list(cross_val_score(classifier, X, y, cv=kf, scoring="accuracy"))

my_classifiers_scores = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, unique_data, 5)
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))

/Users/kaushik/cse40_venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kaushik/cse40_venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

Classifier: LogisticRegression, Accuracy: [np.float64(0.9016393442622951), np.float64(0.8765432098765432), np.float64(0.934156378600823), np.float64(0.8930041152263375), np.float64(0.9012345679012346)].
Classifier: DecisionTreeClassifier, Accuracy: [np.float64(0.7786885245901639), np.float64(0.823045267489712), np.float64(0.8106995884773662), np.float64(0.8024691358024691), np.float64(0.8024691358024691)].
Classifier: KNeighborsClassifier, Accuracy: [np.float64(0.6598360655737705), np.float64(0.6378600823045267), np.float64(0.6995884773662552), np.float64(0.6255144032921811), np.float64(0.6790123456790124)].


/Users/kaushik/cse40_venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


<h4 style="color: darkorange; font-size: x-large";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [26]:
def significance_test(a_values, b_values, p_value):
    t_stat, p_val = ttest_ind(a_values, b_values, equal_var=False)
    return p_val < p_value

for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

LogisticRegression vs DecisionTreeClassifier: True
LogisticRegression vs KNeighborsClassifier: True
DecisionTreeClassifier vs KNeighborsClassifier: True


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table needs a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```

# Part XC: Extra Credit

So far you have used a synthetic dataset that was created just for you.
But, data science is always more interesting when you are dealing with actual data from the real world.
Therefore, you will have an opportunity for extra credit on this assignment using real-world data.

For extra credit, repeat the **written tasks** of Parts 0 through 4 with an additional dataset that you find yourself.
For the written portion of the extra credit for Part 0, include information about where you got the data and what the data represents.
You may choose any dataset that represents real data (i.e., is **not** synthetic or generated)
and is **not** [pre-packaged in scikit-learn](https://scikit-learn.org/stable/datasets.html).

Below are some of the many places you can start looking for datasets:
 - [Kaggle](https://www.kaggle.com/datasets) -- Kaggle is a website focused around machine learning competitions,
       where people compete to see who can get the best results on a dataset.
       It is very popular in the machine learning community and has thousands of datasets with descriptions.
       Make sure to read the dataset's description, as Kaggle also has synthetic datasets.
 - [data.gov](https://data.gov/) -- A portal for data from the US government.
        The US government has a lot of data, and much of it has to be available to the public by law.
        This portal contains some of the more organized data from several different government agencies.
        In general, the government has A LOT of interesting data.
        It may not always be clean (remember the CIA factbook), but it is interesting and available.
        All data here should be real-world, but make sure to read the description to verify.
 - [UCI's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) -- UC Irvine has their own data repository with a few hundred datasets on many different topics.
        Make sure to read the dataset's description, as UCI also has synthetic datasets.
 - [WHO's Global Health Observatory](https://apps.who.int/gho/data/node.home) -- The World Health Organization keeps track of many different health-related statistics for most of the countries in the world.
        All data here should be real-world, but make sure to read the description to verify.
 - [Google's Dataset Search](https://datasetsearch.research.google.com/) -- Google indexes many datasets that can be searched here.

You can even create a dataset from scratch if you find some data you like that is not already organized into a specific dataset.
The only real distinction between "data" and a "dataset" is that a dataset is organized and finite (has a fixed size).

Create a new section in your written report for this extra credit and include all the written tasks for the extra credit there.
Each written task/section that you complete for your new dataset is eligible for extra credit (so you can still receive some extra credit even if you do not complete all parts).
There is no need to submit any code for the extra credit.
If you created a new dataset, include the dataset or links to it with your submission.